# Using a pre-trained model from Huggingface to generate code

This notebook demonstrates how to use a pre-trained model from Huggingface to generate code. The model used in this notebook is `codebert-base` which is a transformer model trained on a large corpus of text data. The model is fine-tuned on the `python` programming language to generate code.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Downloading the model

When we use the model from Huggingface, we do not have to worry about tokenizers and other elements. We can just use the model directly. 


In [2]:
# import the model via the huggingface library
from transformers import AutoTokenizer, AutoModelForMaskedLM

# load the tokenizer and the model for the pretrained SingBERTa
tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')

# load the model
model = AutoModelForMaskedLM.from_pretrained("microsoft/codebert-base")

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Using the right pipeline

Every model has a set of pipelines defined for it, depending on what the model can do. For example, a model that is trained for text generation will have a pipeline for text generation.

The model we are using is a BERT model, which is trained for token classification. We can use the `pipeline` function to get the right pipeline for the model.

In [3]:
# import the pipeline 
from transformers import pipeline

# load the pipeline for text generation
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

# define the code snippet
code_snippet = '''def HelloWorld():
'''
# generate the code
generatedCode = fill_mask(code_snippet + "return '<mask>'")

# print the suggestions for the generated code
generatedCode

2024-03-19 09:09:25.307835: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 09:09:25.309930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 09:09:25.600270: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 09:09:26.323114: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 09:09:34.603122: W tensorflow/compiler/tf2

[{'score': 0.1198003888130188,
  'token': 25741,
  'token_str': 'omew',
  'sequence': "def HelloWorld():\nreturn 'omew'"},
 {'score': 0.10780282318592072,
  'token': 24336,
  'token_str': 'cool',
  'sequence': "def HelloWorld():\nreturn 'cool'"},
 {'score': 0.04159999638795853,
  'token': 33408,
  'token_str': ' discouraging',
  'sequence': "def HelloWorld():\nreturn'discouraging'"},
 {'score': 0.02758779563009739,
  'token': 5715,
  'token_str': 'ood',
  'sequence': "def HelloWorld():\nreturn 'ood'"},
 {'score': 0.02332727238535881,
  'token': 42627,
  'token_str': 'Develop',
  'sequence': "def HelloWorld():\nreturn 'Develop'"}]

## Another model

The model below is Code Parrot. It's not very big, but still can generate code.

In [ ]:
# now, let's do the same for CodeParrot model
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline

# load the tokenizer and the model for the pretrained SingBERTa
tokenizer = AutoTokenizer.from_pretrained('codeparrot/codeparrot')

# load the model
model = AutoModelWithLMHead.from_pretrained("codeparrot/codeparrot")

# load the pipeline for text generation
pipe = pipeline("text-generation", model="codeparrot/codeparrot")

# define the code snippet
code_snippet = '''def HelloWorld():
'''
# generate the code
generatedCode = pipe(code_snippet)

# print the suggestions for the generated code
generatedCode

## Where to go from here

Once you look into these models, you can go into the following directions:
1. Use a different model
    a. Take a look at the `Models for programming languages.md` for a list of other models that are available for programming languages. These models are small enough to run on a laptop and large enough to be useful. 
2. Create a script that will generate function bodies. 
    a. Instead of generating code and that's it, write a function that will parse the `code_snipped` output and truncate it
3. Use the model to generate code based on a prompt
    a. For this task, you need to use a model that has an `instruct` pipeline, e.g., one of the smaller Falcon models, a small Mistral model or a small Llama model. 